# Basic run: How to run NZUpy and customise inputs

## Overview of NZUpy
NZUpy is an optimisation model for simulating the New Zealand Emissions Trading Scheme (NZ ETS). The model simulates interactions between carbon price, emissions, various supply components and the stockpile of units held by private participants.

The model largely repliacates the functionality of the Ministry for the Environment's (MfE's) NZ ETS model that operates in excel (using macros/VBA). However given NZUpy is coded in in Python, it allows for greater flexibility and customisation by users who wish to enhance certain components (e.g., incorporating more sophisticated industrial allocation behaviours)

## Some basics

NZUpy is structured around the `NZUpy` class, which contains all information about the scenario(s), time periods, and components of the model (supply, demand, stockpile dynamics, etc.).

There are four main steps to running NZUpy:

1. **Creating an instance**: Initialise the model
2. **Define the scope**: Specify years, and scenarios
3. **Configure scenarios**: Customise your configuration of inputs and parameters for each scenario
4. **Running the model**: Execute the model and analyse results

The model follows a builder pattern, which allows for flexible configuration.

## 1. Create NZUpy instance

Let's start by importing the NZUpy class and libaries we'll use in the notebook

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import sys
import os

As we're operating in a subfolder off the project root, we'll need to ensure we can find the NZUpy model, and its data directory containing necessary inputs and parameters.

In [2]:
# Add the project root to the path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Import the NZUpy class
from model.core.base_model import NZUpy

# Set our input and output directories
data_dir = project_root / "data"
output_dir = project_root / "examples" / "outputs" / "01_basic_run"
os.makedirs(output_dir, exist_ok=True)

After that, we can initialise NZUpy, pointing it to our input data directory..

In [3]:
NZU = NZUpy(data_dir=data_dir)

## 2. Define the scope

NZUpy requires users to define the time period they wish to model. 

By default, the model is set up for a 2024 start year, and 2050 end year (aligned with MfE's excel-based model), and is recommended for the vast bulk of use cases.

In [ ]:
# Define time periods: start year, end year
NZU.define_time(2024, 2050)

For this run, we'll keep things simple with a single scenario. 

Scenario naming has no effect on the model itself, and is used to enable distinct runs with their own configured settings. So we'll call ours 'Basic run'

In [ ]:
# Define scenarios - you can have multiple scenarios
NZU.define_scenarios(['Basic run'])

We then prime the model, which initialises dataframes housing our variables

In [ ]:
NZU.prime()

## 3. Configure the scenarios

After priming the model, we need to configure our scenario with the input data and parameters we want to use. 

NZUpy makes it easy to operate the modle using central default configured inputs for all of its supply, demand and stockpile components.

We'll use the `use_central_configs()` method to set all components to their central configurations for our scenario.


In [ ]:
# Configure our scenario (index 0) to use central configs for all components
NZU.use_central_configs(0)
# Check stockpile configuration and data
scenario_index = 0
component_config = NZU.component_configs[scenario_index]

print("\nStockpile configuration:")
print(f"Config name: {component_config.stockpile}")

# Get the stockpile parameters
stockpile_params = NZU.data_handler.get_stockpile_parameters(component_config.stockpile)
print("\nInitial balances:")
print(f"Stockpile balance: {stockpile_params['initial_stockpile']:,.0f} kt CO2-e")
print(f"Surplus balance: {stockpile_params['initial_surplus']:,.0f} kt CO2-e")
print(f"Reference year: {stockpile_params['stockpile_reference_year']}")

# Also check other stockpile parameters
print("\nOther stockpile parameters:")
print(f"Liquidity factor: {stockpile_params['liquidity_factor']:.2%}")
print(f"Payback period: {stockpile_params['payback_period']} years")
print(f"Discount rate: {stockpile_params['discount_rate']:.2%}")



In [ ]:
# 1. List available configurations for each component
print("Available configurations by component:")
for component in ['emissions', 'auction', 'industrial', 'forestry', 'demand_model', 'stockpile']:
    configs = NZU.data_handler.list_available_configs(component)
    print(f"\n{component.title()} configs: {configs}")  
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

# 2. List adjustable parameters (single values) for each component
print("\nAdjustable parameters by component (single values):")
params = NZU.data_handler.list_adjustable_parameters()
for component, param_dict in params.items():
    print(f"\n{component.title()} parameters:")
    for param, info in param_dict.items():
        if info.get('type') == 'parameter':  # Only show single-value parameters
            print(f"  - {param}: {info['description']} ({info['units']})")

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

# 3. Show current values for central configuration
print("\nCurrent values for central configuration:")
for component in ['stockpile', 'auction', 'industrial', 'forestry', 'demand']:
    values = NZU.data_handler.show_config_values(component, 'central')
    print(f"\n{component.title()} values:")
    for param, value in values.items():
        if isinstance(value, (int, float)):  # Only show single values
            print(f"  - {param}: {value}")
        else:
            print(f"  - {param}: <time series>")

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

# 4. List available data series (time-varying data)
print("\nAvailable data series by component (time-varying):")
series = NZU.data_handler.list_available_series()
for component, series_dict in series.items():
    print(f"\n{component.title()} series:")
    for series_name, info in series_dict.items():
        print(f"  - {series_name}: {info['description']} ({info['units']})")

We can of course customise these. While the remaining notebooks will show this in more complexity, lets keep it simple

In [ ]:
# List available configs for different components
print("Available emissions configs:", NZU.data_handler.list_available_configs('emissions'))
print("Available auction configs:", NZU.data_handler.list_available_configs('auction'))
print("Available industrial configs:", NZU.data_handler.list_available_configs('industrial'))
print("Available forestry configs:", NZU.data_handler.list_available_configs('forestry'))
print("Available demand model configs:", NZU.data_handler.list_available_configs('demand_model'))
print("Available stockpile balance start configs:", NZU.data_handler.list_available_configs('stockpile'))

